In [228]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import pickle
import time, datetime, os
%reload_ext tensorboard

In [229]:
NAME = "264x32x16x1_MSE_sv{}".format(int(time.time()))
log_dir = "logs/fit/" + NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stats = pd.read_csv(r'Data\final_dataset_nonzero.csv')
#stats = stats.drop('Score',1)
stats['Results'] = pd.to_numeric(stats['Results'], errors='coerce')

C:\Users\Bei\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (269) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [230]:
X = stats.iloc[:,3:-4]
y = stats.iloc[:,-1:]
random_state = 12
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state=random_state)
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)
odds1 = stats.iloc[:, -3:-2].values
odds1_prob = 1/odds1
odds1_prob


array([[0.73529412],
       [0.73529412],
       [0.78740157],
       ...,
       [0.72992701],
       [0.84033613],
       [0.09191176]])

In [231]:
from tensorflow.python.ops import math_ops

def custom_loss_bce(y_true, y_pred):
    """
    y_true = (batch_size, output nodes)
    y_pred = (batch_size, output nodes)
    """
    odds1 = y_true[:, 0:1]
    odds2 = y_true[:, 1:2]
    y_true = y_true[:, :-1]
    loss = -1 * K.sum((odds1 * y_true -1) * K.log(odds1 * y_pred -1) + (odds2 * (1- y_true) -1) * K.log(odds2 * (1-y_pred)-1))
    
    return loss


def coef_mse(y_pred, y_true, odds_game):
    #mse = K.square(y_pred - y_true)  #squared difference
    #odds = 0.05 * K.square(y_pred - odds_game)   
    #loss = K.mean(mse - odds , axis=-1) #mean over last dimension
    return K.mean(K.square(y_pred - y_true) - 0.20 * K.square(y_pred- odds_game), axis=-1)
def mse_custom(odds_game):
    def mse(y_true, y_pred):
        return coef_mse(y_pred, y_true, odds_game )
    return mse


loss_mse = mse_custom(odds1_prob)

def mse_simple(y_true, y_pred):
    mse = math_ops.squared_difference(y_pred, y_true) 
    loss = K.mean(mse, axis=-1)
    return loss
    

In [232]:
#opt = tf.keras.optimizers.Adam(clipnorm=0.3)

model = Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(Dense(264, input_dim=264, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.02)))
model.add(Dense(32, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.02)))
model.add(Dense(32, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.02)))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(optimizer='Adam', loss='MeanSquaredError', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size= 3, epochs=5, validation_data=(X_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/5
3800/3800 [==============================] - 3s 826us/step - loss: 0.2207 - accuracy: 0.6492 - val_loss: 0.2199 - val_accuracy: 0.6552
Epoch 2/5
3800/3800 [==============================] - 3s 759us/step - loss: 0.2164 - accuracy: 0.6611 - val_loss: 0.2158 - val_accuracy: 0.6563
Epoch 3/5
3800/3800 [==============================] - 3s 725us/step - loss: 0.2140 - accuracy: 0.6631 - val_loss: 0.2156 - val_accuracy: 0.6647
Epoch 4/5
3800/3800 [==============================] - 3s 693us/step - loss: 0.2123 - accuracy: 0.6653 - val_loss: 0.2171 - val_accuracy: 0.6626
Epoch 5/5
3800/3800 [==============================] - 3s 696us/step - loss: 0.2107 - accuracy: 0.6714 - val_loss: 0.2156 - val_accuracy: 0.6650


In [233]:
val_loss, val_acc = model.evaluate(X_test, y_test)

90/90 [==============================] - 0s 644us/step - loss: 0.2156 - accuracy: 0.6650


In [234]:
prediction = model.predict(X_test[:])
label = y_test[:]
prediction_l = prediction.tolist()
pred =pd.DataFrame(prediction_l, columns=['prediction'])


In [235]:
stats.reset_index
index = stats[['GAME_DATE', 'HOME', 'AWAY','Odds1', 'Odds2', 'Score']]
data = label.join(index)
data.reset_index(inplace=True)
pred_data = data.join(pred)
pred_data.to_csv('predicted_data_custom.csv')